In [1]:
def extEucl(m, n):
    """
    Возвращает тройку (d,u,v) таких, что d — наибольший общий делитель пары (m,n) и d=um+vn
    """
    (a, b) = (m, n)
    u1 = 1; v1 = 0
    u2 = 0; v2 = 1

    while b != 0:
        assert (a == u1*m + v1*n and b == u2*m + v2*n)

        q = a // b; r = a % b
        assert (a == q*b + r)

        (a, b) = (b, r)
        (u1, u2) = (u2, u1 - q*u2)
        (v1, v2) = (v2, v1 - q*v2)

    if a >= 0:
        return (a, u1, v1)
    else:
        return (-a, -u1, -v1)
    
def invmod(x, m):
    """
    Возвращает обратный к x элемент, если x обратим (т.е. целые числа x,m взаимно простые),
    иначе - исключение типа ValueError
    """
    assert(m != 0)
    (d, u, v) = extEucl(m, x)
    if d == 1:
        return v%m
    else:
        raise ValueError("Element is not invertible modulo m")
        
def calc_comparison(h,m,a=1):
    """
    Решение сравнения: a*x≡h(mod m)
    """
    q=[]
    (a, b) = (m, a)
    c=m
    while b != 0:
        (a, b) = (b, a%b)     
        q.append((c-b)//a)
        c=a
        
    P=[0 for i in range(len(q))]
    for i in range(len(q)):
        if i==0:
            P[0]=q[0]
        elif i==1:
            P[1]=q[1]*P[0]+1
        else:
            P[i]=q[i]*P[i-1]+P[i-2]
            
    return (((-1)**(len(q)-1))*P[-2]*h)%m

## Алиса выбирает открытые и закрытые параметры
* q - большое положительное целое число
* f - целое число, удовлетворяющее условию: $$f<\sqrt{\frac{q}{2}}$$
* g - целое число, удовлетворяющее условию: $$\sqrt{\frac{q}{4}}<g<\sqrt{\frac{q}{2}}$$

Так же, f и g подбираются таким образом, что:
$$(f,qg) = 1$$

In [2]:
q = 122430513841
f = 231231
g = 195698

Вычисляем $$f^{-1}$$ по модулю **q**

In [3]:
f_i=invmod(f,q)
assert f_i==49194372303

Вычисляем $$h \equiv f^{−1}g \pmod{q}$$

In [4]:
h=(g*f_i)%q
assert h==39245579300

##### Открытый ключ Алисы - это пара (q, h). Закрытый - (f, g)
## Очередь Боба
* m - исходное сообщение в виде числа такое, что: $$0<m<\sqrt{\frac{q}{4}}$$
* r - случайное целое число, такое что: $$0<r<\sqrt{\frac{q}{2}}$$

In [5]:
m = 123456
r = 101010

Шифруем исходное сообщение с помошью открытого ключа Алисы: $$e \equiv rh+m \pmod{q}$$ причём $$0 < e < q$$

In [6]:
e=(r*h+m)%q
assert e==18357558717

##### Передаём **e** по каналу связи
## Алиса принимает сообщение
Вычисляем $$a \equiv fe\pmod{q}$$ причём $$0 < a < q$$ 

In [7]:
a=(f*e)%q
assert a==48314309316

Вычисляем $$f^{-1}$$ по модулю **g**

In [8]:
f_i=invmod(f,g)
assert f_i==193495

Вычисляем $$b \equiv f^{-1}a\pmod{g}$$ причём $$0 < b < g$$

In [9]:
b=(f_i*a)%g
assert b==123456

##### Готово! Мы получили исходное сообщение.
## Вмешивается Ева
Её задача найти **F** и **G** такие, что: $$F(1, h) − R(0, q) =(F, G)$$ где 
* (1,h) и (0,q) - известные вектора
* F, R - неизвестные целые
* (F, G) - неизвестныый малый вектор
##### Таким образом, Еве нужно найти короткий ненулевой вектор в наборе векторов $$L = a_1v_1 + a_2v_2 : a_1, a_2 ∈ Z $$

In [10]:
import numpy as np
import math

def Gaussian_Lattice_Reduction(a,b):
    v1,v2=a.copy(),b.copy()
    norma= lambda v:math.sqrt(v[0]**2 + v[1]**2)
    while True:
        if norma(v2)<norma(v1):
            v1,v2=v2,v1
        m=round(np.inner(v1,v2)/(norma(v1)**2))
        if m==0:
            return v1,v2
        v2=v2-m*v1

In [11]:
v1,v2=np.array([1,h],dtype="float_"),np.array([0,q],dtype="float_")
v1_,v2_=Gaussian_Lattice_Reduction(v1,v2)
print(f"v1:\n{v1}\nv2:\n{v2}")
print(f"v1_:\n{v1_}\nv2_:\n{v2_}")

v1:
[1.00000000e+00 3.92455793e+10]
v2:
[0.00000000e+00 1.22430514e+11]
v1_:
[-231231. -195698.]
v2_:
[-368222.  217835.]


In [30]:
norma= lambda v:math.sqrt(v[0]**2 + v[1]**2)
if norma(v1_)<norma(v2_):
    private_keys=v1_
else:
    private_keys=v2_
private_keys=np.abs(private_keys)

In [31]:
find_f=np.max(private_keys)
find_g=np.min(private_keys)

In [32]:
a=(f*e)%q
f_i=invmod(f,g)
print("Взломанное сообщение:",(f_i*a)%g)

Взломанное сообщение: 123456


## Функциональная реализация

In [46]:
def encypt(m,r,h,q):
    return (r*h+m)%q

def decrypt(e,h,q,f,g):
    a=(f*e)%q
    f_i=invmod(f,g)
    return (f_i*a)%g

def hak_with_Gauss(e,h,q):
    v1,v2=Gaussian_Lattice_Reduction(np.array([1,h],dtype="float_"),np.array([0,q],dtype="float_"))
    norma= lambda v:math.sqrt(v[0]**2 + v[1]**2)
    if norma(v1)<norma(v2):
        private_keys=v1
    else:
        private_keys=v2
    private_keys=np.abs(private_keys)
    find_f=np.max(private_keys)
    find_g=np.min(private_keys)
    
    a=(find_f*e)%q
    f_i=invmod(find_f,find_g)
    return int((f_i*a)%find_g)

In [47]:
m_t = 128986 # сообщение
r_t = 101010 # случайное число

f_t = 231231 # секретный ключ Алисы
g_t = 195698 # секретный ключ Алисы
q_t = 122430513841 # открытый ключ Алисы
h_t = 39245579300 # открытый ключ Алисы

In [48]:
e_t=encypt(m_t,r_t,h_t,q_t)
d_t=decrypt(e_t,h_t,q_t,f_t,g_t)
hak_t=hak_with_Gauss(e_t,h_t,q_t)

print("Исходное:",m_t,"\nЗашифрованное:",e_t,"\nРасшифрованное:",d_t,"\nВзоманное:",hak_t)

Исходное: 128986 
Зашифрованное: 18357564247 
Расшифрованное: 128986 
Взоманное: 128986
